In [1]:
# 1. 환경 세팅 + 스모크테스트

!pip -q install pybullet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# 셀 A — 드라이브 마운트 & 경로 추가 & 모듈 임포트

from google.colab import drive
drive.mount('/content/drive')

import sys, os, math
# 👉 네 실제 경로를 하나만 남기고 사용해
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Robot_Simulation')


from src.core.sim import connect, base_pose, step_n
from src.core.kinematics import v_omega_to_wheels
from src.core.controllers import set_wheel_angular

# (선택) 시각화 유틸 쓸 거면
# from src.viz.plot2d import plot_path_with_heading
# from src.viz.anim import animate_path

print("OK: imports loaded.")


Mounted at /content/drive
OK: imports loaded.


**주의점**: 1. child_link의 실제(world) 위치 = parent_link 위치 ⊕ joint.origin ⊕ link.visual.origin (⊕는 “좌표변환 합성(translation + rotation)”)

2. x축 기준 중심축 회전 = x축을 직교축으로 해서 해당 중심축을 직교축 기준으로 회전시키는 것 목표축과 기준축을 헷갈리지 말기


**<URDF 설정간 알아야 하는 개념들>**

1. 여기서 v = 바퀴의 선속도 (m/s), R = 바퀴 반지름 (m).

바퀴가 땅에 굴러갈 때:

**𝑣=𝜔⋅𝑅**


→ 즉, **바퀴 각속도 ω(rad/s)**와 **바퀴 선속도 v(m/s)**는 반지름으로 연결돼.


2. **v_l, v_r 공식(v_l = v - (L/2)·ω, v_r = v + (L/2)·ω)**

이건 차동구동 로봇의 핵심 운동학 공식이야.

로봇이 전진 속도 v (m/s), 회전 각속도 ω (rad/s)로 움직인다고 하자.

그러면 왼쪽 바퀴 속도 v_l, 오른쪽 바퀴 속도 v_r는 이렇게 정해져:

𝑣𝑙=𝑣−𝐿/2⋅𝜔

𝑣𝑟=𝑣+𝐿/2⋅𝜔


여기서 L = 바퀴 사이 거리(트랙폭).

왜 이런 공식이 나오냐?

로봇이 회전할 때, 바깥쪽 바퀴는 더 멀리, 안쪽 바퀴는 더 짧게 이동해야 원을 그림.

예를 들어 L=0.15 m, ω=2 rad/s 라면:

왼쪽 바퀴는 중앙보다 반쪽(0.075 m) 안쪽 원을 돌고,

오른쪽 바퀴는 0.075 m 바깥 원을 돌아야 해.

그래서 속도를 다르게 줘야 하는데, 그 차이가 ± (L/2)·ω로 계산되는 거




3. **캐스터(caster)란**?

우리가 타는 의자나 쇼핑카트 바닥 보면, 앞/뒤에 작은 회전하는 보조 바퀴 달려 있지? → 그게 캐스터야.

차동구동 로봇(바퀴 2개로 움직이는 로봇)은 사실 바퀴 2개만 있으면 균형 못 잡음 → 뒤쪽이 땅에 쓸리거나 넘어져.

그래서 보통은 뒤에 마찰 거의 없는 작은 바퀴(캐스터) 하나 달아서 균형만 잡고, 구동에는 쓰지 않아.

URDF에서 캐스터는 fixed로 붙여뒀지? 이유는 시뮬 단순화 때문이야. 실제 물리 구현까지 하려면 caster wheel도 회전 joint 달아야 하는데, 굳이 그럴 필요 없이 “뒤쪽에 땅에 닿는 점만 있음” 정도로만 표현해도 잘 굴러.



4. **로봇에서 회전 각속도**

차동구동 로봇은 왼/오른쪽 바퀴 속도를 다르게 주면 빙글빙글 돌지?
이때 로봇 본체가 중심축 기준으로 얼마나 빨리 돌아가는지를 **회전각속도 𝜔** 라고 해.


ω>0: 반시계방향 회전 (왼쪽으로 돈다)
 by 오른손 법칙(반시계 = 왼쪽으로 돔이 양수)

ω<0: 시계방향 회전 (오른쪽으로 돈다)

In [3]:
# 셀 B — 환경 스모크(URDF 경로 확인 + 연결/해제)

try:
    import src.core.sim as sim
    print("sim.URDF_PATH =", getattr(sim, "URDF_PATH", None))
    # 필요시 이렇게 재설정 (런타임 동안만)
    # sim.URDF_PATH = "/content/drive/MyDrive/Colab Notebooks/Robot_Simulation/urdf/diff_drive_min.urdf"
except Exception as e:
    print("sim import err:", e)

# 연결 테스트
import pybullet as p
if p.isConnected(): p.disconnect()
robot, jl, jr = connect() # Removed gui=False
print(f"[OK] robot loaded. joints: L={jl}, R={jr}")

# joint 이름 확인(문제시 디버그)
for j in range(p.getNumJoints(robot)):
    print(j, p.getJointInfo(robot, j)[1].decode())

p.disconnect()
print("[OK] disconnected.")

sim.URDF_PATH = /content/drive/MyDrive/Colab Notebooks/Robot_Simulation/urdf/diff_drive_min.urdf
[OK] robot loaded. joints: L=0, R=1
0 joint_wheel_left
1 joint_wheel_right
2 joint_caster
[OK] disconnected.


In [5]:
#셀 C — 직선 주행 스모크 (v=0.2, 1초)

if p.isConnected(): p.disconnect()
robot, jl, jr = connect(gui=False)

# (선택) 타임스텝 고정
try:
    import src.core.sim as sim
    p.setTimeStep(sim.DT)
except: pass

# 안착
set_wheel_angular(robot, jl, jr, 0.0, 0.0, max_torque=0.0)
step_n(0.7 / (1/240))

v, omega = 0.2, 0.0
w_l, w_r = v_omega_to_wheels(v, omega)
pos0, yaw0 = base_pose(robot)
set_wheel_angular(robot, jl, jr, w_l, w_r, max_torque=60.0)
step_n(1.0 / (1/240))
pos1, yaw1 = base_pose(robot)

# 정지
set_wheel_angular(robot, jl, jr, 0.0, 0.0, max_torque=0.0)
step_n(0.2 / (1/240))
p.disconnect()

dist = ((pos1[0]-pos0[0])**2 + (pos1[1]-pos0[1])**2)**0.5
print("[Straight]")
print("start:", pos0, "yaw:", round(math.degrees(yaw0),2))
print("end  :", pos1, "yaw:", round(math.degrees(yaw1),2))
print("dist ~", round(dist, 3), "m (theory ~0.2)")


[Straight]
start: (-0.0005913833176002186, -1.663523592660776e-06, 0.06098526447374848) yaw: -0.0
end  : (0.1983625045284475, -6.683231389928842e-06, 0.06085570980196948) yaw: -0.0
dist ~ 0.199 m (theory ~0.2)


In [6]:
# 셀 D — 제자리 180° + 원호(반지름 0.5, 3초)

if p.isConnected(): p.disconnect()
robot, jl, jr = connect(gui=False)
try:
    import src.core.sim as sim
    p.setTimeStep(sim.DT)
except: pass

# 안착
set_wheel_angular(robot, jl, jr, 0.0, 0.0, max_torque=0.0)
step_n(0.7 / (1/240))

# (a) 제자리 회전
omega_turn = math.pi/2
w_l, w_r = v_omega_to_wheels(0.0, omega_turn)
pos0, yaw0 = base_pose(robot)
set_wheel_angular(robot, jl, jr, w_l, w_r, max_torque=80.0)
step_n((math.pi/omega_turn) / (1/240))  # 2초
posA, yawA = base_pose(robot)

# (b) 원호: r=0.5, v=0.2 → ω=0.4, t=3
v_arc, r_arc = 0.2, 0.5
omega_arc = v_arc/r_arc
w_l, w_r = v_omega_to_wheels(v_arc, omega_arc)
set_wheel_angular(robot, jl, jr, w_l, w_r, max_torque=80.0)
posB0, yawB0 = base_pose(robot)
step_n(3.0 / (1/240))
posB1, yawB1 = base_pose(robot)

# 끝
set_wheel_angular(robot, jl, jr, 0.0, 0.0)
step_n(0.2 / (1/240))
p.disconnect()

def deg(x): return round(math.degrees(x),2)
print("[Turn-in-place 180°] start:", deg(yaw0), "→ end:", deg(yawA))
print("[Arc] start:", tuple(round(v,4) for v in posB0), "yaw:", deg(yawB0))
print("      end  :", tuple(round(v,4) for v in posB1), "yaw:", deg(yawB1),
      " Δyaw~", deg(yawB1-yawB0))


[Turn-in-place 180°] start: -0.0 → end: 0.26
[Arc] start: (-0.0596, -0.0001, 0.0609) yaw: 0.26
      end  : (0.5116, 0.0073, 0.0608) yaw: 1.22  Δyaw~ 0.96


오차원인: 처음 로봇 시작 조금 떠서 시작 -> 내려오는 시간 필요, 바퀴 돌리는 토크힘 부족 -> 목표 속도 도달 시간 존재

In [7]:
# 셀 E — 사각형(0.25×4) + 클로징 에러

import pybullet as p
if p.isConnected(): p.disconnect()
robot, jl, jr = connect(gui=False)
try:
    import src.core.sim as sim
    p.setTimeStep(sim.DT)
except: pass

def drive_straight(distance_m, v=0.18, tq=80.0):
    t = abs(distance_m)/max(1e-6, abs(v))
    wl, wr = v_omega_to_wheels(v if distance_m>=0 else -v, 0.0)
    set_wheel_angular(robot, jl, jr, wl, wr, max_torque=tq)
    step_n(t / (1/240))
    set_wheel_angular(robot, jl, jr, 0.0, 0.0)
    step_n(0.1 / (1/240))

def rotate_deg(angle_deg, ang_vel=math.pi/2, tq=80.0):
    ang = math.radians(angle_deg)
    t = abs(ang)/max(1e-6, abs(ang_vel))
    wl, wr = v_omega_to_wheels(0.0, ang_vel if ang>=0 else -ang_vel)
    set_wheel_angular(robot, jl, jr, wl, wr, max_torque=tq)
    step_n(t / (1/240))
    set_wheel_angular(robot, jl, jr, 0.0, 0.0)
    step_n(0.1 / (1/240))

# 안착
set_wheel_angular(robot, jl, jr, 0.0, 0.0)
step_n(0.7 / (1/240))

pos0, yaw0 = base_pose(robot)
for _ in range(4):
    drive_straight(0.25, v=0.18, tq=80.0)
    rotate_deg(90, ang_vel=math.pi/2, tq=80.0)
pos1, yaw1 = base_pose(robot)

set_wheel_angular(robot, jl, jr, 0.0, 0.0)
step_n(0.2 / (1/240))
p.disconnect()

dx, dy = pos1[0]-pos0[0], pos1[1]-pos0[1]
closing_err = (dx*dx + dy*dy)**0.5
print("[Square 0.25×4] closing error:", round(closing_err,3), "m (target < 0.2)")


[Square 0.25×4] closing error: 0.95 m (target < 0.2)


In [ ]:
!pip -q install imageio imageio-ffmpeg

import numpy as np, imageio
from math import radians

W, H = 640, 480   # 출력 해상도
FPS = int(1/DT)   # PyBullet 스텝이 240Hz라면 대충 240fps; 파일 용량 줄이려면 30으로 바꿔도 됨
FRAMES = []

def topdown_view(center=(0,0,0.05), yaw_deg=0.0, dist=1.5):
    """월드 고정 탑뷰(원점 주변 시야). 필요하면 center를 로봇 위치로 따라가게 바꿔도 됨."""
    cx, cy, cz = center
    view = p.computeViewMatrixFromYawPitchRoll(
        cameraTargetPosition=[cx, cy, cz],
        distance=dist, yaw=45, pitch=-70, roll=0, upAxisIndex=2
    )
    proj = p.computeProjectionMatrixFOV(fov=60, aspect=W/H, nearVal=0.01, farVal=10.0)
    return view, proj

def follow_robot_view(robot, dist=1.2, pitch=-30, yaw_offset_deg=0):
    """로봇을 따라다니는 3rd-person 뷰. (더 역동적)"""
    (x,y,z), yaw = base_pose(robot)
    view = p.computeViewMatrixFromYawPitchRoll(
        cameraTargetPosition=[x, y, z],
        distance=dist,
        yaw=np.degrees(yaw) + yaw_offset_deg,
        pitch=pitch, roll=0, upAxisIndex=2
    )
    proj = p.computeProjectionMatrixFOV(fov=60, aspect=W/H, nearVal=0.01, farVal=10.0)
    return view, proj

def grab_frame(view, proj):
    img = p.getCameraImage(W, H, view, proj, renderer=p.ER_TINY_RENDERER)
    rgba = np.reshape(img[2], (H, W, 4))
    rgb = rgba[...,:3]
    FRAMES.append(rgb)

def step_and_capture(n_steps, view_fn):
    """시뮬레이션 n스텝 진행하면서 매 스텝 프레임 저장(너무 크면 간격 캡처로 변경 가능)"""
    for _ in range(int(n_steps)):
        p.stepSimulation()
        view, proj = view_fn()
        grab_frame(view, proj)


In [ ]:
# (재)연결
try:
    if p.isConnected(): p.disconnect()
except: pass
robot, jl, jr = connect()
set_wheel_angular(robot, jl, jr, 0.0, 0.0, max_torque=0.0)
step_n(int(0.5/DT))

# 촬영용: 탑뷰 또는 팔로우 뷰 중 하나 선택
# view_fn = lambda: topdown_view(center=(0,0,0.05), dist=1.8)           # 월드 고정 탑뷰
view_fn = lambda: follow_robot_view(robot, dist=1.2, pitch=-35, yaw_offset_deg=30)  # 로봇 따라가는 뷰

# 직선/회전을 캡처 버전으로 다시 정의
def drive_straight_cap(distance_m, v=0.18, torque=80.0):
    t = abs(distance_m)/max(1e-6, abs(v))
    w_l, w_r = v_omega_to_wheels(v if distance_m>=0 else -v, 0.0)
    set_wheel_angular(robot, jl, jr, w_l, w_r, max_torque=torque)
    step_and_capture(int(t/DT), view_fn)
    set_wheel_angular(robot, jl, jr, 0.0, 0.0)
    step_and_capture(int(0.2/DT), view_fn)

def rotate_deg_cap(angle_deg, ang_vel=np.pi/2, torque=80.0):
    angle = radians(angle_deg)
    w_l, w_r = v_omega_to_wheels(0.0, ang_vel if angle>0 else -ang_vel)
    set_wheel_angular(robot, jl, jr, w_l, w_r, max_torque=torque)
    step_and_capture(int(abs(angle)/max(1e-6,abs(ang_vel)) / DT), view_fn)
    set_wheel_angular(robot, jl, jr, 0.0, 0.0)
    step_and_capture(int(0.2/DT), view_fn)

# 시작 포즈 저장
pos0, yaw0 = base_pose(robot)

# ▶ 사각형 주행 촬영
for _ in range(4):
    drive_straight_cap(0.25, v=0.18, torque=80.0)
    rotate_deg_cap(90, ang_vel=np.pi/2, torque=80.0)

pos1, yaw1 = base_pose(robot)
set_wheel_angular(robot, jl, jr, 0.0, 0.0)
step_and_capture(int(0.3/DT), view_fn)

# 종료
p.disconnect()

# 리포트
dx, dy = pos1[0]-pos0[0], pos1[1]-pos0[1]
closing_err = (dx*dx+dy*dy)**0.5
print("[Square path 0.25m × 4]")
print("start:", tuple(round(v,4) for v in pos0), "yaw(deg):", round(np.degrees(yaw0),2))
print("end  :", tuple(round(v,4) for v in pos1), "yaw(deg):", round(np.degrees(yaw1),2))
print("closing error:", round(closing_err, 3), "m   (target < 0.2 m)")


[Square path 0.25m × 4]
start: (-0.0005, -0.0, 0.061) yaw(deg): -0.02
end  : (-0.0795, 0.2143, 0.0609) yaw(deg): -63.85
closing error: 0.228 m   (target < 0.2 m)


In [ ]:
# (가벼움) GIF
imageio.mimsave('/content/square_run.gif', FRAMES, fps=30)
print("Saved: /content/square_run.gif")

Saved: /content/square_run.gif


In [ ]:
!ls -lh /content

total 5.9M
-rw-r--r-- 1 root root 3.0K Oct  8 13:59 diff_drive_min.urdf
drwxr-xr-x 1 root root 4.0K Oct  6 13:38 sample_data
-rw-r--r-- 1 root root 5.9M Oct  8 14:08 square_run.gif


In [ ]:
from google.colab import files
files.download('/content/square_run.gif')   # 또는 .gif


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>